In [1]:
!pip install ultralytics --upgrade -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
from ultralytics import YOLO

extract_dir = '/content/drive/MyDrive/yolo_data'
data_yaml_path = os.path.join(extract_dir, "data.yaml")


with open(data_yaml_path, "w") as f:

    f.write("""path: /content/drive/MyDrive/yolo_data
train: train
val: valid
test: test

names:
  0: fracture
""")


with open(data_yaml_path, "r") as f:
    print(f.read())


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
path: /content/drive/MyDrive/yolo_data
train: train
val: valid
test: test

names:
  0: fracture



In [4]:
def evaluate_and_print_metrics(model, split_name):
    """
    Runs evaluation on a given data split and prints key metrics.

    Args:
        model (YOLO): The trained YOLO model.
        split_name (str): The name of the data split to evaluate ('train', 'val', or 'test').
    """
    print(f"\n Running evaluation on the '{split_name}' set...")
    try:
        metrics = model.val(split=split_name, project='runs/eval', name=f'{split_name}_set_metrics')

        print(f"\n Results for '{split_name}' set:")


        if not hasattr(metrics, 'box') or metrics.box.map is None:
            print(" Could not compute metrics for this split. Is the dataset empty?")
            return

        if metrics.box.p.size > 0:

            accuracy_precision = metrics.box.p[0]

            precision = metrics.box.p[0]

            recall = metrics.box.r[0]

            f1 = metrics.box.f1[0]

            map50_95 = metrics.box.map

            map50 = metrics.box.map50

            print(f"   ┃ Accuracy (Precision): {accuracy_precision:.3f}")
            print(f"   ┃ F1 Score            : {f1:.3f}")
            print(f"   ┃ Recall              : {recall:.3f}")
            print(f"   ┃ mAP50-95            : {map50_95:.3f}")
            print(f"   ┃ mAP50               : {map50:.3f}")
        else:
            print("   ┃  No objects detected or ground truth available in this split.")

    except Exception as e:
        print(f"    An error occurred during '{split_name}' set evaluation: {e}")


def train_and_evaluate_yolo_model():
    """
    Loads, trains, and comprehensively evaluates a YOLOv8 model.
    """
    try:
        model = YOLO('yolov8n.pt')
        print(" YOLOv8n base model loaded.")
    except Exception as e:
        print(f" Model load error: {e}")
        return

    print("\n Starting model training...")
    try:
        model.train(
            data=data_yaml_path,
            project='runs/train',
            name='bone_fracture_final',
            epochs=80,
            imgsz=416,
            batch=16,
            workers=8,
            patience=5
        )
        save_dir = model.trainer.save_dir
        print(f"Training complete!")
        print(f" Model and results saved to: {save_dir}")
        print("-" * 40)
    except Exception as e:
        print(f" Training failed with error: {e}")
        return


    evaluate_and_print_metrics(model, 'train')


    evaluate_and_print_metrics(model, 'val')


    evaluate_and_print_metrics(model, 'test')


if __name__ == '__main__':
    train_and_evaluate_yolo_model()

100%|██████████| 6.25M/6.25M [00:00<00:00, 362MB/s]


 YOLOv8n base model loaded.

 Starting model training...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bone_fracture_final, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto

100%|██████████| 755k/755k [00:00<00:00, 118MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all       4947       6513      0.879      0.767      0.843      0.437
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/eval/train_set_metrics

 Results for 'train' set:
   ┃ Accuracy (Precision): 0.879
   ┃ F1 Score            : 0.819
   ┃ Recall              : 0.767
   ┃ mAP50-95            : 0.437
   ┃ mAP50               : 0.843

 Running evaluation on the 'val' set...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 26.9±8.6 MB/s, size: 52.6 KB)


val: Scanning /content/drive/MyDrive/yolo_data/valid/labels.cache... 472 images, 0 backgrounds, 0 corrupt: 100%|██████████| 472/472 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:06<00:00,  4.29it/s]


                   all        472        590      0.871      0.707      0.786      0.409
Speed: 0.5ms preprocess, 2.9ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/eval/val_set_metrics

 Results for 'val' set:
   ┃ Accuracy (Precision): 0.871
   ┃ F1 Score            : 0.780
   ┃ Recall              : 0.707
   ┃ mAP50-95            : 0.409
   ┃ mAP50               : 0.786

 Running evaluation on the 'test' set...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.0±0.0 MB/s, size: 37.7 KB)


val: Scanning /content/drive/MyDrive/yolo_data/test/labels.cache... 235 images, 0 backgrounds, 0 corrupt: 100%|██████████| 235/235 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.07it/s]


                   all        235        306      0.878      0.703      0.802       0.39
Speed: 1.7ms preprocess, 3.5ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/eval/test_set_metrics

 Results for 'test' set:
   ┃ Accuracy (Precision): 0.878
   ┃ F1 Score            : 0.780
   ┃ Recall              : 0.703
   ┃ mAP50-95            : 0.390
   ┃ mAP50               : 0.802
